In [66]:
%pip install pandas
%pip install tabula-py

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:

import os
import tabula
import pandas as pd
import re
from datetime import datetime, timedelta

directory = 'data/uk'

pdf_files = [f for f in os.listdir(directory) if f.endswith('.pdf')]

# Do not run this loop as it will overwrite the manually edited csv files
# for pdf_file in pdf_files:
#     pdf_file_name = re.sub('.pdf', '', pdf_file)
#     file_path = os.path.join(directory, pdf_file)
#     tabula.convert_into(file_path, f"{directory}/{pdf_file_name}.csv", output_format="csv", pages='all')
#     print(f"Converted {pdf_file} to csv")

In [15]:
def rename_columns(columns):
    column_mapping = {
        re.compile(r'date', re.IGNORECASE): 'date',
        re.compile(r'time', re.IGNORECASE): 'time',
        re.compile(r'(town|village)', re.IGNORECASE): 'city',
        re.compile(r'county', re.IGNORECASE): 'state',
        re.compile(r'occupation', re.IGNORECASE): 'reporters occupation',
        re.compile(r'description', re.IGNORECASE): 'description'
    }
    new_columns = []
    for col in columns:
        for pattern, new_name in column_mapping.items():
            if pattern.search(col):
                new_columns.append(new_name)
                break
        else:
            new_columns.append(col)  # Keep the original column name if no pattern matches

    # Ensure the columns are unique
    unique_columns = []
    seen = set()
    for col in new_columns:
        if col in seen:
            counter = 1
            new_col = f"{col}_{counter}"
            while new_col in seen:
                counter += 1
                new_col = f"{col}_{counter}"
            unique_columns.append(new_col)
            seen.add(new_col)
        else:
            unique_columns.append(col)
            seen.add(col)

    return unique_columns


In [16]:
def correct_datetime(dt_str):
    # Check if the time is "24:00" and adjust accordingly
    if dt_str.endswith("24:00"):
        # Extract date part and convert it to datetime object
        date_part = dt_str.split()[0]
        date_obj = datetime.strptime(date_part, "%m/%d/%Y")
        # Increment the day by one
        corrected_date_obj = date_obj + timedelta(days=1)
        # Format back to string and replace "24:00" with "00:00"
        corrected_dt_str = corrected_date_obj.strftime("%m/%d/%Y") + " 00:00"
        return corrected_dt_str
    else:
        return dt_str

In [17]:
all_csv_files = [f for f in os.listdir(directory) if re.search(r'\.csv$', f)]
# print(all_csv_files)
combined_csv_data = pd.DataFrame()

for csv_file in all_csv_files:
    csv_file_path = os.path.join(directory, csv_file)
    try:
        temp_df = pd.read_csv(csv_file_path, on_bad_lines='skip')
        # Drop unnamed columns
        # Identify and remove columns that start with 'Unnamed'
        temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed')]
        # Alternatively, to remove columns starting specifically with 'Unnamed: '
        temp_df = temp_df.loc[:, ~temp_df.columns.str.startswith('Unnamed: ')]
        temp_df.columns = rename_columns(temp_df.columns)  # Rename the columns
        combined_csv_data = pd.concat([combined_csv_data, temp_df], ignore_index=True)
    except pd.errors.ParserError as e:
        print(f"Error parsing {csv_file_path}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred with {csv_file_path}: {e}")
combined_csv_data['country'] = 'gb'
combined_csv_data.to_csv(os.path.join('data', 'uk_ufo_sightings.csv'), index=False)
combined_csv_data.head(5)


,date,time,city,Area,reporters occupation,description,state,country
0,Jan-09,Not Given,Warwick,Warwickshire,Air Traffic Control\remployee,An orange glowing object with a red light on t...,NaN,gb
1,Jan-09,Not Given,Not Given,North Yorkshire,NaN,"Very curious. A very bright light, four times ...",NaN,gb
2,Jan-09,Not Given,Carterton,Oxfordshire,Air Traffic Control\remployee,A very bright constant red light.,NaN,gb
3,Jan-09,Not Given,High Wycombe,Buckinghamshire,NaN,been outside the individuals house for some ni...,NaN,gb
4,Jan-09,Not Given,Sawtry,Cambridgeshire,NaN,A UFO.,NaN,gb


In [18]:
uk = pd.read_csv('data/uk_ufo_sightings.csv')
# uk.head(10)
uk.shape

print(uk['country'].dropna().unique())

['gb']


In [19]:
us = pd.read_csv('data/us_ufo_sightings.csv')

# Convert the 'date_time' column to datetime using this formatting 10/10/1949 20:30
us['date_time'] = us['date_time'].apply(correct_datetime)
us['date_time'] = pd.to_datetime(us['date_time'], format="%m/%d/%Y %H:%M")

# Extract the date and time from the 'DateTime' column
us['date'] = us['date_time'].dt.date
us['time'] = us['date_time'].dt.time

us = us.drop(columns=['date_time'])

us.rename(columns={'city_area': 'city'}, inplace=True)

us.head(10)
# us.shape

,city,state,country,ufo_shape,encounter_length,described_encounter_length,description,date_documented,latitude,longitude,date,time
0,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,1949-10-10,20:30:00
1,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,1949-10-10,21:00:00
2,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,1955-10-10,17:00:00
3,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,1956-10-10,21:00:00
4,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,1960-10-10,20:00:00
5,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,1961-10-10,19:00:00
6,penarth (uk/wales),NaN,gb,circle,180.0,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000,1965-10-10,21:00:00
7,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333,1965-10-10,23:45:00
8,pell city,al,us,disk,180.0,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.586111,-86.286111,1966-10-10,20:00:00
9,live oak,fl,us,disk,120.0,several minutes,Saucer zaps energy from powerline as my pregna...,5/11/2005,30.294722,-82.984167,1966-10-10,21:00:00


In [23]:
ufo_sightings = pd.concat([us, uk], axis=0, join='outer', ignore_index=True)
ufo_sightings.to_csv('data/ufo_sightings.csv', index=False)
ufo_sightings.shape
ufo_sightings.head(10)

,city,state,country,ufo_shape,encounter_length,described_encounter_length,description,date_documented,latitude,longitude,date,time,Area,reporters occupation
0,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,1949-10-10,20:30:00,NaN,NaN
1,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,1949-10-10,21:00:00,NaN,NaN
2,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,1955-10-10,17:00:00,NaN,NaN
3,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,1956-10-10,21:00:00,NaN,NaN
4,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,1960-10-10,20:00:00,NaN,NaN
5,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,1961-10-10,19:00:00,NaN,NaN
6,penarth (uk/wales),NaN,gb,circle,180.0,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000,1965-10-10,21:00:00,NaN,NaN
7,norwalk,ct,us,disk,1200.0,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333,1965-10-10,23:45:00,NaN,NaN
8,pell city,al,us,disk,180.0,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.586111,-86.286111,1966-10-10,20:00:00,NaN,NaN
9,live oak,fl,us,disk,120.0,several minutes,Saucer zaps energy from powerline as my pregna...,5/11/2005,30.294722,-82.984167,1966-10-10,21:00:00,NaN,NaN
